In [ ]:
import pandas as pd
import numpy as np

from ortools.linear_solver import pywraplp

import random

pd.set_option('display.max_rows', 50)


from functions import get_tickers_stocks, get_close_prices, double_listed_stocks, sharpe_ratio_calculation, generate_rand_portfolios, select_top_five

In [4]:
#GET THE STOCKS

us_exchanges = ['NMS', 'NYQ', 'NGM']
eu_exchanges = ['PAR', 'FRA', 'LSE', 'AMS']
asia_exchanges = ['SHH', 'JPX', 'HKG']

selected_exchanges = us_exchanges + eu_exchanges + asia_exchanges

full_selected_stocks = {}
df_all_stocks = pd.DataFrame()
for exchange in selected_exchanges:
    print(f'Extracting from {exchange}')
    exchanges = [exchange]
    selected_stocks_dict, ticker_list = get_tickers_stocks(50000, exchanges, 50)

    full_selected_stocks.update(selected_stocks_dict)

    if len(ticker_list) > 0: 
        print('YES')
        df = get_close_prices(ticker_list, period = 2, start = '2022-01-01')
        df_all_stocks = pd.concat([df_all_stocks, df], axis=1)

doubly_listed_tickers = double_listed_stocks(full_selected_stocks)

for ticker_to_drop in doubly_listed_tickers:
    try:
        df_all_stocks = df_all_stocks.drop(columns=[ticker_to_drop])
    except:
        pass


df_all_stocks = df_all_stocks.ffill() #ffill again after concatenating the tickers

Extracting from NMS


[*****                 10%                       ]  5 of 50 completed

YES
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  50 of 50 completed

1 Failed download:
['KSPI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1641013200, endDate = 1704085200")')
[**                     4%                       ]  2 of 50 completed

Extracting from NYQ
YES


...

Extracting from JPX
YES


[*********************100%***********************]  50 of 50 completed

1 Failed download:
['6701.T']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01)')
[                       0%                       ]

Extracting from HKG
YES


[*********************100%***********************]  49 of 50 completed

2 Failed downloads:
['0300.HK', '6936.HK']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1640966400, endDate = 1704038400")')


There is no name found in dict for SOUNW


In [ ]:
#GETTING CRYPTOS FROM COINBASE 50 INDEX
#https://www.marketvector.com/factsheets/download/COIN50.d.pdf

coinbase_50_cryptos = ['BTC', 'ETH', 'XRP', 'SOL', 'DOGE', 'ADA', 'LINK', 'XLM', 'AVAX', 'SHIB', 'DOT', 'LTC', 'BCH', 
                       'UNI', 'NEAR', 'PEPE', 'APT', 'ICP', 'ETC', 'AAVE', 'RNDR', 'ATOM', 'MATIC', 'ALGO', 'EOS', 'MKR', 
                       'ASI', 'QNT', 'BONK', 'STX', 'INJ', 'GRT', 'LDO', 'XTZ', 'CRV', 'SAND', 'ZEC', 'HNT', 'JASMY', 'MANA', 
                       'AXS', 'WIF', 'CHZ', 'COMP', 'APE', 'AERO', '1INCH', 'SNX', 'ROSE', 'LPT']

crypto_tickers_fixed = [tick + "-USD" for tick in coinbase_50_cryptos]

cryptos_df = get_close_prices(crypto_tickers_fixed, period = 2, start = '2022-01-01')

print('Number of NA values in cryptos_df is ', df.isna().any().sum())

#cryptos_df

[*********************100%***********************]  50 of 50 completed

1 Failed download:
['ASI-USD']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1640995200, endDate = 1704067200")')


Number of NA values in cryptos_df is  0


Ticker,1INCH-USD,AAVE-USD,ADA-USD,ALGO-USD,APE-USD,APT-USD,ATOM-USD,AVAX-USD,AXS-USD,BCH-USD,...,SAND-USD,SHIB-USD,SNX-USD,SOL-USD,STX-USD,UNI-USD,XLM-USD,XRP-USD,XTZ-USD,ZEC-USD
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,2.530239,265.499847,1.376975,1.740108,0.033021,0.084735,36.170464,114.158012,94.042458,444.456177,...,5.972598,0.000034,5.971597,178.517944,2.283105,0.000143,0.276335,0.849472,4.740467,149.951370
2022-01-02,2.555007,266.026184,1.377584,1.674041,0.033103,0.084714,35.603455,113.186333,96.511070,447.832642,...,5.930911,0.000034,6.240839,176.382843,2.391267,0.000142,0.291703,0.859118,4.667668,153.354340
2022-01-03,2.559547,267.053955,1.321637,1.788210,0.030884,0.082153,39.300068,108.145210,94.323700,434.943848,...,5.687358,0.000033,7.129435,170.297745,2.297845,0.000139,0.290181,0.832885,4.802002,150.387634
2022-01-04,2.489328,252.842041,1.311658,1.674683,0.030788,0.090146,41.987125,103.239243,93.001984,427.524323,...,5.475225,0.000032,6.602596,167.938904,2.348068,0.000138,0.279667,0.824673,5.003270,147.749680
2022-01-05,2.332959,231.419067,1.236002,1.559897,0.032450,0.097028,38.793358,97.852577,80.350739,401.572510,...,5.018268,0.000030,6.414337,155.099731,2.244129,0.000261,0.267280,0.774358,4.528895,137.754776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,0.447588,112.113907,0.633535,0.229226,1.797234,0.001969,11.965517,42.410416,9.727940,264.086853,...,0.603856,0.000011,4.163719,107.023796,0.005286,0.000217,0.130853,0.634856,1.112585,31.771292
2023-12-28,0.428446,118.968285,0.617422,0.234213,1.721247,0.002115,11.429394,39.906868,9.107651,262.496643,...,0.580764,0.000011,4.062906,102.104568,0.005230,0.000213,0.132362,0.634998,1.055397,31.906683
2023-12-29,0.431516,112.426804,0.608494,0.226423,1.746465,0.004423,10.991527,40.211086,9.095331,254.512726,...,0.574967,0.000010,3.893410,106.311516,0.005570,0.000211,0.129619,0.623402,1.031104,28.686319


In [6]:
#Random Portfolios generation

tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [14]:
#Select top five sharpe ratio portfolios from a portfolio

sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)

top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [8]:
####HERE GOES OR TOOLS OPTIMIZATION

# Example covariance matrix for 3 assets (just for illustration)
cov_matrix = np.array([[0.0004, 0.0002, 0.0001],
                       [0.0002, 0.0005, 0.0003],
                       [0.0001, 0.0003, 0.0006]])

# Number of assets
n_assets = len(cov_matrix)

# Create the solver using the GLOP solver (for linear programming)
solver = pywraplp.Solver.CreateSolver('GLOP')

if not solver:
    print('Solver not created!')
    exit()

# Variables: asset weights (w1, w2, ..., wn)
weights = []
for i in range(n_assets):
    weights.append(solver.IntVar(0.0, 1.0, f'weight_{i}'))

# Constraint: fully invested (sum of weights = 1)
solver.Add(solver.Sum(weights[i] for i in range(n_assets)) == 1)

# Objective function: minimize portfolio variance
objective = solver.Objective()

# Portfolio variance = w^T * Covariance Matrix * w
for i in range(n_assets):
    for j in range(n_assets):
        objective.SetCoefficient(weights[i], cov_matrix[i][j])

# Objective direction: minimize (we're minimizing variance)
objective.SetMinimization()

# Solve the problem
status = solver.Solve()

# Check the result
if status == pywraplp.Solver.OPTIMAL:
    print('Optimal solution found:')
    for i in range(n_assets):
        print(f'Weight of asset {i}: {weights[i].solution_value()}')
else:
    print('No optimal solution found')



Optimal solution found:
Weight of asset 0: 1.0
Weight of asset 1: 0.0
Weight of asset 2: 0.0
